In [1]:
import os

In [2]:
%pwd

'd:\\documents\\Code\\InceptionBD\\End-to-End-Deep-Learning-Chicken-Disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\documents\\Code\\InceptionBD\\End-to-End-Deep-Learning-Chicken-Disease'

In [15]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [6]:
import os
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= self.config.model_trainer.trained_model_path,
            training_data= os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [17]:
import tensorflow as tf
from cnnClassifier import logger
from cnnClassifier.utils.common import save_json

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        # Load the already trained model for evaluation
        self.model = tf.keras.models.load_model(self.config.path_of_model)
        logger.info(f"Model loaded from path: {self.config.path_of_model}")


    def _valid_generator(self):
        # Create validation data generator (same as training pipeline but only for validation)
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30     # 30% of data is used for validation here
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        # Only validation data is needed here; no training data or augmentation
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",       # Use validation split of data
            shuffle=False,             # No need to shuffle when evaluating
            **dataflow_kwargs
        )


    def evaluation(self):
        # Generate validation data and evaluate model performance
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  # Returns [loss, accuracy]


    def save_score(self):
        # Save evaluation score to a JSON file
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        logger.info(f"Scores saved at path: scores.json")


In [21]:
try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-11-04 08:07:38,809: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-04 08:07:38,813: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-04 08:07:38,815: INFO: common: created directory at: artifacts]
[2025-11-04 08:07:39,415: INFO: 3617826342: Model loaded from path: artifacts/trained_model/model.h5]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 21s 3s/step - loss: 5.2917 - accuracy: 0.5000
[2025-11-04 08:08:00,713: INFO: common: json file saved at: scores.json]


In [ ]:
#Test cell
model = tf.keras.models.load_model("artifacts/trained_model/model.h5")
print(model.summary())
print(model.metrics_names)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     